## FINAL PROJECT SUBMISSION

- GROUP 5

STUDENT NAME:

**- OLIVE MULOMA**

**- ABIGAIL MWENDWA**

**- HAWKINS MURITHI**

**- HARRY ATULAH**

STUDENT PACE: *FULL TIME/HYBRID*

SCHEDULE PROJECT REVIEW DATE/TIME: *29TH JULY 2024*

INSTRUCTOR NAME: **MARYANN MWIKALI**


# NEWSNET PUBLISHING COMPANY

## BUSINESS OVERVIEW

- In today's digital age, news consumption has skyrocketed, and with it, the challenge of efficiently managing and categorizing vast amounts of content. Automated text classification offers a solution by enabling efficient sorting of news articles into relevant categories such as Politics, Technology, Entertainment, and Business. This enhances user experience by ensuring that readers have quick access to content that interests them and assists in organizing and managing content for media companies.

## IMPORT REQUIRED LIBRARIES

In [2]:
import pandas as pd 
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from itertools import cycle
from sklearn.preprocessing import label_binarize
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import string
import pickle
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from flask import Flask, request, jsonify, render_template_string
import os
import warnings
warnings.filterwarnings('ignore')

## DATA UNDERSTANDING

In [3]:
data_test = pd.read_csv("data/Data_Test.csv", encoding='latin1')

data_test.head(10)

,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...
5,"""Imagine if every message you sent was kept wi..."
6,Positioned along the four sides of the Asus RO...
7,"In fact, when I applied to USC film school the..."
8,"As spotted by Android Police, Netflix is testi..."
9,Her moves were immaculately choreographed as s...


In [4]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   STORY   2748 non-null   object
dtypes: object(1)
memory usage: 21.6+ KB


#### The DataFrame `data_test` has:

- All entries in the "STORY" column are non-null and contain text data
- The DataFrame occupies 21.6 KB of memory

**- Description:**  The test dataset contains 2,748 rows and 1 column: STORY.


**- STORY:** Contains the text of news articles, similar to the training dataset but without associated category labels.


**-Usage:** This dataset is used to evaluate the performance of the trained model. The STORY column will be input to the model to predict the category labels, which can then be compared to actual labels (if available) or used for making predictions on new data.

In [5]:
data_test.shape

(2748, 1)

In [6]:
data_train = pd.read_csv("data/Data_Train.csv", encoding='latin1')

data_train.head(10)

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ...",1
4,"In global markets, gold prices edged up today ...",3
5,BEIJING: Chinese tech giant Huawei has announc...,1
6,Mumbai: India Inc's external commercial borrow...,3
7,"On Wednesday, Federal Reserve Chairman Jerome ...",3
8,What more can you give to the audience? I have...,2
9,"com, Arbaaz Khan spoke about getting back to D...",2


In [7]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   STORY    7628 non-null   object
 1   SECTION  7628 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 119.3+ KB


#### The DataFrame `data_train` has:
    

- All entries in both columns are non-null
- The "STORY" column contains text data (`object` type)
- The "SECTION" column contains integer data (`int64` type)
- The DataFrame occupies 119.3 KB of memory

**- Description:** The training dataset contains 7,628 rows and 2 columns: STORY and SECTION.


**- STORY:** Contains the text of news articles, which will be used to train the model.


**- SECTION:** Contains numerical labels representing different categories or topics (e.g., 0 for Politics, 1 for Technology, etc.), which are the target labels for classification.

**- Usage:** - This dataset is used to train the text classification model. The STORY column provides the content for the model to learn from, while the SECTION column serves as the ground truth labels for supervised learning.

In [8]:
data_train.shape

(7628, 2)

## DATA CLEANING

### Checking for Duplicates

In [9]:
duplicates = data_train[data_train.duplicated()]

duplicates

,STORY,SECTION
587,"Facebook, WhatsApp and Twitter have overhauled...",1
757,We will leave no stone unturned to make the au...,2
948,"""Marvel's 616,"" a documentary series exploring...",1
1472,"Facebook, WhatsApp and Twitter have overhauled...",1
1831,We will leave no stone unturned to make the au...,2
...,...,...
7453,"The whole feels like a giant set, stately and ...",2
7462,"""Monsters at Work,"" a series inspired by Pixar...",1
7465,Commenting on the upcoming Amazon summer sale ...,1
7474,Congress says it needs to verify whether there...,0


- We check for duplicate rows in the data_train DataFrame.In this case, it identified 77 duplicate rows with repeated STORY and SECTION values.

In [10]:
data = data_train.drop_duplicates()

data

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ...",1
4,"In global markets, gold prices edged up today ...",3
...,...,...
7623,"Karnataka has been a Congress bastion, but it ...",0
7624,"The film, which also features Janhvi Kapoor, w...",2
7625,The database has been created after bringing t...,1
7626,"The state, which has had an uneasy relationshi...",0


- We then remove the duplicate rows identified in the previous step from the DataFrame.
- The resulting DataFrame (data) now contains only unique rows. It reduced the total number of rows from 7628 to 7551, removing 77 duplicates.

### Adding and Renaming Columns